# BİLGİSAYAR MÜHENDİSLİĞİ - MAKİNE ÖĞRENMESİ PROJE ÖDEVİ
**Adınız:** Eren Ali Koca  
**Okul Numaranız:** 2212721021  
**GitHub Repo Bağlantısı:** https://github.com/erenalikoca/YoloV8_Nesne_Tespiti  

Bu notebook Google Colab'da çalıştırılmak üzere hazırlanmıştır.


## 1. Google Drive Bağlantısı

Google Drive'ınızı bağlayın ve izin verin.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import os
os.chdir('/content/drive/MyDrive/Uyg2')
print("Çalışma dizini:", os.getcwd())


## 2. GPU Kontrolü

Colab'da GPU kullandığınızdan emin olun: Runtime → Change runtime type → GPU


In [ ]:
!nvidia-smi


## 3. Kütüphane Kurulumu

YOLOv8 için Ultralytics kütüphanesini yüklüyoruz.


In [ ]:
!pip install ultralytics

import ultralytics
from ultralytics import YOLO
from IPython.display import Image
import shutil

ultralytics.checks()


## 4. Veri Seti Yapısı

Google Drive'ınızda şu klasör yapısını oluşturun:

```
MyDrive/
└── Uyg2/
    ├── data.yaml
    └── yolo_dataset/
        ├── train/
        │   ├── images/  (103 görsel)
        │   └── labels/  (103 txt)
        └── val/
            ├── images/  (27 görsel)
            └── labels/  (27 txt)
```

Veri setimizde **2 sınıf** bulunmaktadır: `airpods` ve `magic_mouse`


## 5. Model Eğitimi

YOLOv8n (nano) modelini transfer learning ile eğitiyoruz.

**Eğitim Parametreleri:**
- **data**: `/content/drive/MyDrive/Uyg2/data.yaml`
- **epochs**: 50
- **imgsz**: 640
- **device**: 0 (GPU)


In [ ]:
model = YOLO('yolov8n.pt')

results = model.train(
    data='/content/drive/MyDrive/Uyg2/data.yaml',
    epochs=50,
    imgsz=640,
    device=0,
    plots=True,
    name='yolov8_custom_training'
)


## 6. Eğitim Sonuçları

Loss ve mAP grafiklerini görselleştiriyoruz.


In [ ]:
results_path = f'{results.save_dir}/results.png'
if os.path.exists(results_path):
    display(Image(filename=results_path))
else:
    print("Grafik bulunamadı.")


## 7. Confusion Matrix

Modelin sınıflar arası performansını gösteren confusion matrix'i görüntüleyelim.


In [ ]:
confusion_path = f'{results.save_dir}/confusion_matrix.png'
if os.path.exists(confusion_path):
    display(Image(filename=confusion_path))


## 8. Model Performans Değerlendirmesi

Doğrulama seti üzerinde modelin performansını ölçüyoruz.


In [ ]:
metrics = model.val()
print(f"mAP50: {metrics.box.map50:.4f}")
print(f"mAP50-95: {metrics.box.map:.4f}")


## 9. Eğitilmiş Modelin Kaydedilmesi

En iyi model ağırlıkları Drive'a kaydedilir.


In [ ]:
best_model_path = f'{results.save_dir}/weights/best.pt'
destination_path = '/content/drive/MyDrive/Uyg2/best.pt'

if os.path.exists(best_model_path):
    shutil.copy(best_model_path, destination_path)
    print(f"✅ Model kaydedildi: {destination_path}")
else:
    print("❌ Model dosyası bulunamadı.")


## 10. Test Görseli ile Deneme

Eğitilen modeli bir test görseli üzerinde deneyelim.


In [ ]:
test_image = '/content/drive/MyDrive/Uyg2/yolo_dataset/val/images/IMG_6103.JPG'

results = model(test_image)
results[0].show()
